In [ ]:
import numpy as np
import pymannkendall as pmk
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download
from dask.diagnostics import ProgressBar

In [ ]:
# Time
year_start = 1959
year_stop = 1969

# Models
models = [
    "access_cm2",
    "awi_esm_1_1_lr",
    "bcc_esm1",
    "cesm2_fv2",
    "cnrm_cm6_1",
    "fgoals_g3",
    "mpi_esm1_2_lr",
    "hadgem3_gc31_ll",
]

# Chunks for download
chunks = {"year": 1}

In [ ]:
common_request = {
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 12 + 1)],
}

request_era = (
    "reanalysis-era5-single-levels-monthly-means",
    {
        "product_type": "monthly_averaged_reanalysis",
        "format": "netcdf",
        "time": "00:00",
        "variable": "mean_total_precipitation_rate",
        **common_request,
    },
)

request_sim = (
    "projections-cmip6",
    {
        "format": "zip",
        "temporal_resolution": "monthly",
        "experiment": "historical",
        "variable": "precipitation",
        **common_request,
    },
)

In [ ]:
def original_test(data, attrs):
    test = pmk.original_test(data)
    return np.array([getattr(test, attr) for attr in attrs])


def compute_mannkendall_trends(ds):
    # Varname
    varname = {"mtpr", "pr"} & set(ds.data_vars)
    assert len(varname) == 1, varname
    varname, *_ = varname

    # Annual weighted mean
    ds = diagnostics.annual_weighted_mean_timeseries(ds)

    # Compute trendes
    attrs = ["p", "slope"]
    delayed_obj = xr.apply_ufunc(
        original_test,
        ds[varname],
        attrs,
        input_core_dims=[["year"], ["_tmp_dim"]],
        output_core_dims=[["_tmp_dim"]],
        exclude_dims={"year"},
        vectorize=True,
        dask="parallelized",
        output_dtypes=[float],
        dask_gufunc_kwargs={
            "output_sizes": {"_tmp_dim": len(attrs)},
            "allow_rechunk": True,
        },
    )
    with ProgressBar():
        da = delayed_obj.compute()

    # Return dataset
    return da.assign_coords(_tmp_dim=attrs).to_dataset("_tmp_dim")

In [ ]:
datasets = {}
for model in models:
    print(f"Downloading and processing {model}")
    request_model = request_sim
    request_model[1]["model"] = model
    ds = download.download_and_transform(
        *request_model,
        chunks=chunks,
        transform_func=compute_mannkendall_trends,
        transform_func_kwargs={},
        transform_chunks=False,
    )
    datasets[model] = ds


print("Downloading and processing ERA5")
datasets["ERA5"] = download.download_and_transform(
    *request_era,
    chunks=chunks,
    transform_func=compute_mannkendall_trends,
    transform_chunks=False,
)